In [1]:
#extract sequences from NCBI BLAST hit table
#filter sequences by e-value & best hit (unique sequence)
import pandas as pd
from Bio import SeqIO
df = pd.read_csv('GCE32a_pol_hit_table.csv')
print(df.head())

#e-value cutoff 1e-10

def extract_coord(i, df):
    accession = str(df.loc[i, 'Subject'])
    start = str(df.loc[i, 'Subject_start'])
    end = str(df.loc[i, 'Subject_end'])
    return accession, start, end

grandlist = []
for i in range(len(df.index)):
    if i == 0:  #first special case
        accession, start, end = extract_coord(i, df)
        grandlist.append([accession, int(start)-10000, int(end)+10000])
    elif str(df.loc[i,'Subject']) != str(df.loc[i-1,'Subject']):
        value = str(df.loc[i, 'E-value'])
        if value == '0':
            accession, start, end = extract_coord(i, df)
            grandlist.append([accession, int(start)-10000, int(end)+10000])
        elif 'e' in value[-5:]:
            if value[-3] == '1' or value[-3] == '2':
                accession, start, end = extract_coord(i, df)
                grandlist.append([accession, int(start)-10000, int(end)+10000])
            elif not value[-2] == '0':
                accession, start, end = extract_coord(i, df)
                grandlist.append([accession, int(start)-10000, int(end)+10000])

print(len(grandlist))
print(grandlist[0])

#fetch the sequences with efetch
import os
import subprocess
i = 0
for seq in grandlist:
    if seq[1] > 0:
        subprocess.run('efetch -db nucleotide -id {} -seq_start {} -seq_stop {} -format fasta > {}_{}.fasta'.format(seq[0], seq[1], seq[2], seq[0], seq[1]), shell=True)
    i += 1
    if i % 50 == 0:
        print(i)

              Query         Subject  Identity  Length  Mismatch  Gap_opens  \
0  GCE32a_consensus  NW_023416309.1    88.931     262        29          0   
1  GCE32a_consensus  NW_023416309.1    78.986     138        26          1   
2  GCE32a_consensus  NW_023416309.1    77.419      62        14          0   
3  GCE32a_consensus  NW_023416309.1    81.250      48         9          0   
4  GCE32a_consensus  NW_023416309.1    49.174     242       120          3   

   Query_start  Query_end  Subject_start  Subject_end        E-value  \
0            1        262       32241761     32240976  1.160000e-130   
1            1        138       16365067     16364663   1.430000e-84   
2          191        252       16364508     16364323   1.430000e-84   
3          149        196       16364633     16364490   1.430000e-84   
4           14        254       65859429     65860148   1.210000e-47   

   bit_score  Positives  Q_frame  S_frame  
0      419.0      92.75        0       -1  
1      186

In [1]:
#merge all the files
import os
from Bio import SeqIO
dir = os.getcwd()
filelist = []
for file in os.listdir(dir):
    filename = os.fsdecode(file)
    if filename.endswith(".fasta"):
        filelist.append(filename)
        continue
    else: 
        continue
print(len(filelist))
i = 0

for file in filelist:
    for seq_record in SeqIO.parse('{}'.format(file), 'fasta'):
        with open('combined.fasta', 'a') as output_handle:
            output_handle.write('>' + str(seq_record.description) + '\n' + str(seq_record.seq) + '\n')
        i += 1
        if i % 100 == 0:
            print(i)

478
100
200
300
400
